<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Software-and-conventions" data-toc-modified-id="Software-and-conventions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Software and conventions</a></span></li><li><span><a href="#Compling-the-code" data-toc-modified-id="Compling-the-code-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Compling the code</a></span></li></ul></li><li><span><a href="#Overview-of-Inputs" data-toc-modified-id="Overview-of-Inputs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Overview of Inputs</a></span><ul class="toc-item"><li><span><a href="#Basin-Metadata" data-toc-modified-id="Basin-Metadata-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Basin Metadata</a></span></li><li><span><a href="#Establishing-Parameters-to-Calibrate" data-toc-modified-id="Establishing-Parameters-to-Calibrate-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Establishing Parameters to Calibrate</a></span></li><li><span><a href="#The-Calibration-Configuration-File" data-toc-modified-id="The-Calibration-Configuration-File-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>The Calibration Configuration File</a></span></li><li><span><a href="#logistics-section" data-toc-modified-id="logistics-section-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>logistics section</a></span></li><li><span><a href="#Sensitivity" data-toc-modified-id="Sensitivity-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Sensitivity</a></span></li><li><span><a href="#gageInfo" data-toc-modified-id="gageInfo-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>gageInfo</a></span></li><li><span><a href="#lsmPhysics" data-toc-modified-id="lsmPhysics-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>lsmPhysics</a></span></li><li><span><a href="#forcing" data-toc-modified-id="forcing-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>forcing</a></span></li><li><span><a href="#modelTime" data-toc-modified-id="modelTime-2.9"><span class="toc-item-num">2.9&nbsp;&nbsp;</span>modelTime</a></span></li><li><span><a href="#hydroIO" data-toc-modified-id="hydroIO-2.10"><span class="toc-item-num">2.10&nbsp;&nbsp;</span>hydroIO</a></span></li><li><span><a href="#hydroPhysics" data-toc-modified-id="hydroPhysics-2.11"><span class="toc-item-num">2.11&nbsp;&nbsp;</span>hydroPhysics</a></span></li></ul></li><li><span><a href="#The-calibration-workflow" data-toc-modified-id="The-calibration-workflow-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>The calibration workflow</a></span></li><li><span><a href="#Database-initialization" data-toc-modified-id="Database-initialization-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Database initialization</a></span></li><li><span><a href="#Domain-information" data-toc-modified-id="Domain-information-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Domain information</a></span></li><li><span><a href="#Experiment-initialization" data-toc-modified-id="Experiment-initialization-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Experiment initialization</a></span></li><li><span><a href="#Execution-of-calibration" data-toc-modified-id="Execution-of-calibration-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Execution of calibration</a></span></li></ul></div>

# Lesson - NWM calibration

## Overview

In this lesson, we will run a hands-on calibration experiment with the Poconos watershed. Specifically, we will be calibrating the basin using observed and modeled streamflow at gage 01447720. This calibration exercise will focus on a short duration in September, 2011 coinciding with Hurricane Irene. We will be running a 25 iteration calibration exercise. 

### Software and conventions

To run the calibration workflow, there are multiple software requirements and pre-processing steps that need to take place prior to actual calibration. Currently, the calibration workflow relies on a set of Python and R libraries to execute various functions and plotting capabilities. This is in addition to existing requirements to compile and run WRF-Hydro/NWM. As of early 2019, the calibration workflow code has been migrated from Python 2 to Python 3. The following Python libraries are used by the workflow:

1.) numpy
2.) NetCDF4
3.) pandas
4.) psutil

For R, there are several additional libraries required for proper analysis and plotting to take place. In the future, functions in R will be migrated over to Python to reduce dependencies. The following R libraries are needed for proper execution:

1.) data.table
2.) ggplot2
3.) ncdf4
4.) plyr
5.) boot
6.) sensitivity
7.) randtoolbox
8.) gridExtra

### Compling the code
We will recompile NWM with spatial soils on, and nudging off since we are not doing any nudging with obs. 

In [ ]:
%%bash
cd /home/docker/nwm-training/wrf_hydro_nwm_public/trunk/NDHMS
cp ~/nwm-training/wrf_hydro_nwm_public/trunk/NDHMS/template/setEnvar.sh \
~/nwm-training/wrf_hydro_nwm_public/trunk/NDHMS/calibEnvars.sh
sed -i -e 's#export SPATIAL_SOIL=0#export SPATIAL_SOIL=1#g' ~/nwm-training/wrf_hydro_nwm_public/trunk/NDHMS/calibEnvars.sh
./configure 2
./compile_offline_NoahMP.sh calibEnvars.sh >> compile_calib.log 2>&1
tail -13 ~/nwm-training/wrf_hydro_nwm_public/trunk/NDHMS/compile_calib.log

## Overview of Inputs

Let's take a high-level look at the inputs needed to run the code. 

In [ ]:
%%bash
cd /home/docker/nwm-training/example_case/Calibration
ls

In [ ]:
%%bash
cd /home/docker/nwm-training/example_case/Calibration/Input_Files
ls

In [ ]:
%%bash
cd /home/docker/nwm-training/example_case/Calibration/Input_Files/01447720
ls -lh

In [ ]:
%%bash
cd /home/docker/nwm-training/example_case/Calibration/Input_Files/01447720/OBS
ls

### Basin Metadata

For each basin being calibrated, you must specify information about the basin domain that helps guide the calibration workflow to the proper input files, including forcings and observations. The domainMetaTemplate.csv is a template file that user can fill in for each of their domains they are calibrating. You can put multiple domains within one csv file and enter the information all at once. For our exercise, we will be filliing out the template file for our Poconos domain. This fille is then ingested into the workflow to store information in database tables. 

In [ ]:
%%bash
cd /home/docker/nwm-training/example_case/Calibration
cat domainMetaTemplate.csv

### Establishing Parameters to Calibrate

For each calibration experiment, you must point the workflow to a table of parameter values you wish to calibrate. The calib_params.tbl lists the parameters for the model that can be modified. Some parameters are spatially distributed, while others are simply in a table file. The user can turn calibration for various parameters on/off by adjusting the calib_flag to either 1/0. There are also min/max/default values that specify the parameter space the workflow will be searching for, and adjusting parameter values. 

In [ ]:
%%bash
cd /home/docker/nwm-training/example_case/Calibration
cat calib_params.tbl

### The Calibration Configuration File

Lastly, to run a calibration experiment, you must fill out a configuration file that specifies defines your calibration experiment. This includes date ranges for your model simulations, output directories, physics options, etc. Below is a list of the configuration options. We will be going more into your specific configuration file below...
### logistics section
**outDir** - Req. Where do you want your calibration experiment to be constructed? <br>
**expName** - Req. What is the name of your experiment. This can be anything you want. <br>
**acctKey** - Opt. If you are running on a que system that requires credentials to submit a job, specify your account key here. <br>
**optQueName** - Opt. If you need to direct model simulations/jobs to a specific que, you can specify that here. <br>
**nCoresModel** - Req. How many CPUs are you running your model simulations over? <br>
**nNodesModel** - Req. If you are running across multiple nodes on a que system, specify here.<br> 
**nCoresPerNode** - Req. If you are running on a que system, specify how many CPU cores you have available per node. <br>
**runSens** - Req. Are we running sensitivity analysis? 0 - No, 1 - Yes <br>
**sensParmTbl** - Req. Path to the table of parameter values to use in sensitivity analysis. Same format as the calibration parameter table. <br>
**runCalib** - Req. Are we running calibration? 0 - No, 1 - Yes <br>
**calibParmTbl** - Req. Path to the table of parameter values to use in calibration. <br>
**dailyStats** - Req. Flag to direct worfklow to calculate error statistics on a daily scale, instead of hourly by default. Specify 1 to activate. <br>
**coldStart** - Req. Flag to direct calibration workflow to cold start your model simulation for each iteration during calibration. Specify 1 to activate. <br>
**optSpinFlag** - Req. Flag to direct the workflow to use an alternative spinup file already in place in the input directory for the basins being calibrated. This allows the user to bypass the spinup step. Specify 1 to activate. <br>
**stripCalibOutputs** - Req. If you desire to ommit outputs during an intial window for each model iteration, you can specify 1 here to activate this feature. This was designed to minimize I/O model burdens. <br>
**stripCalibHours** - Req. Specify an initial window in hours to strip outputs. This is only used if stripCalibOutputs has been activated. <br>
**jobRunType** - Req. How are the model simulations and calibration code being executed? For this exercise we will be using 4, which is MPI with no job scheduler. <br>
**mpiCmd** - Req. What is the MPI command being used to execute the model simulations. This is required, as the MPI command is also used in job scheduler scripts. <br>
**cpuPinCmd** - Opt. If you are running on a job scheduler, how do you want to pin specific model simulations on a compute node? This put in to allow multiple basins on one node. <br>
**numIter** - Req. How many model iterations would you like to run your calibration experiment over? <br>
**calibMethod** - Req. Right now only DDS is allowed. Future upgrades will incorporate additional calibration methods. <br>
**objectiveFunction** - Req. What error metric do you wish to minimize during the calibraion experiment. <br>
**ddsR** - Req. This is a DDS-specific parameter that tunes how random values are generated for each iteration. <br>
**email** - Req. Where do you want status and error messages to be directed to? <br>
**wrfExe** - Req. Path to the WRF-Hydro executable to be used in the workflow. <br>
**genParmTbl** - Req. Path to the GENPARM.TBL file used by the model. <br>
**mpParmTbl** - Req. Path to the MPARM.TBL file used by the model. <br>
**urbParmTbl** - Req. Path to the URBPARM.TBL file used by the model. <br>
**vegParmTbl** - Req. Path to the VEGPARM.TBL file used by the model. <br>
**soilParmTbl** - Req. Path to the SOILPARM.TBL file used by the model. <br>
**bSpinDate** - Opt. Beginning date for the spinup. <br>
**eSpinDate** - Opt. Ending date for the spinup. <br>
**bCalibDate** - Req. Beginning date for each calibration iteration. <br>
**eCalibDate** - Req. Ending date for each calibration iteration. <br>
**bCalibEvalDate** - Req. The date within each calibration iteration to begin analysis. <br>
**bValidDate** - Opt. Beginning date for the validation simulation. <br>
**eValidDate** - Opt. Ending date for the validation simulation. <br>
**bValidEvalDate** - Opt. The date within the validation simulation to begin analysis. <br>

### Sensitivity
**sensParmSample** - Opt. Sensitivity parameter. <br>
**sensBatchNum** - Opt. How many sensitivity simulations to run at once. <br>
**bSensDate** - Opt. Beginning date for the sensitivity model simulation. <br>
**eSensDate** - Opt. Ending date for the sensitivity model simulation. <br>
**bSensEvalDate** - Opt. The date within the sensitivity simulation to begin analysis. <br>

### gageInfo
**gageListSQL** - Req. SQL command to extrat basins for calibration out of the database file. <br>
**gageListFile** - Opt. Alternative list of basins to calibrate instead of using an SQL command. <br>

### lsmPhysics
**dynVegOption** - Req. DYNAMIC_VEG_OPTION for NoahMP. <br>
**canStomResOption** - Req. CANOPY_STOMATAL_RESISTANCE_OPTION for NoahMP. <br>
**btrOption** - Req. BTR_OPTION for NoahMP. <br>
**runoffOption** - Req. RUNOFF_OPTION for NoahMP. <br>
**sfcDragOption** - Req. SURFACE_DRAG_OPTINO for NoahMP. <br>
**frzSoilOption** - Req. FROZEN_SOIL_OPTION for NoahMP. <br>
**supCoolOption** - Req. SUPERCOOLED_WATER_OPTION for NoahMP. <br>
**radTransferOption** - Req. RADIATIVE_TRANSFER_OPTION for NoahMP. <br>
**snAlbOption** - Req. SNOW_ALBEDO_OPTION for NoahMP. <br>
**pcpPartOption** - Req. PCP_PARTITION_OPTION. <br>
**tbotOption** - Req. TBOT_OPTINO for NoahMP. <br>
**tempTimeScOption** - Req. TEMP_TIME_SCHEME_OPTION for NoahMP. <br>
**sfcResOption** - Req. SURFACE_RESISTANCE_OPTION for NoahMP. <br>
**glacierOption** - Req. GLACIER_OPTION for NoahMP. <br>
**soilThick** - Req. Soil thicknesses for specified soil layers in NoahMP. <br>
**zLvl** - Req. Level of wind speeds in NoahMP. <br>

### forcing
**forceType** - Req. Specified forcing type. <br>

### modelTime
**forceDt** - Req. Input forcing timestep in seconds. <br>
**lsmDt** - Req. NoahMP timestep in seconds. <br>
**lsmOutDt** - Req. NoahMp output timestep in seconds. <br>
**lsmRstFreq** - Req. NoahMp restart frequency in seconds. <br>
**hydroRstFreq** - Req. WRF-Hydro restart frequency in seconds. <br>
**hydroOutDt** - Req. WRF-Hydro output frequency in seconds. <br>

### hydroIO
**rstType** - Req. Flag for overwritting accumulation vars in restart file. <br>
**ioConfigOutputs** - Req. Output flag for varible grouping in WRF-Hydro. <br>
**ioFormOutputs** - Req. Flag to for specifying output format. <br>
**chrtoutDomain** - Req. Flag to turn on CHRTOUT_DOMAIN files. <br>
**chanObsDomain** - Req. Flag to turn on CHANOBS_DOMAIN files. <br>
**chrtoutGrid** - Req. Flag to turn on CHRTOUT_GRID files. <br>
**lsmDomain** - Req. Flag to turn on LSMOUT_DOMAIN files. <br>
**rtoutDOmain** - Req. Flag to turn on RTOUT_DOMAIN files. <br>
**gwOut** - Req. Flag to turn on GWOUT_DOMAIN files. <br>
**lakeOut** - Req. Flag to turn on LAKEOUT_DOMAIN files. <br>
**frxstOut** - Req. Flag to turn on FRXST output text files. <br>
**resetHydroAcc** - Req. Flag to reset accumulation variables in the restart files. <br>
**streamOrderOut** - Req. Flag to specify the minimum Strahler order to output. <br>

### hydroPhysics
**dtChSec** - Req. Channel routing timesetp in seconds. <br>
**dtTerSec** - Req. Surface and subsurface routing timestep in seconds. <br>
**subRouting** - Req. Flag to turn on/off subsurface routing. <br>
**ovrRouting** - Req. Flag to turn on/off overland flow routing. <br>
**channelRouting** - Req. Flag to turn on/off channel routing. <br>
**rtOpt** - Req. Overland/subsurface routing option. <br>
**chanRtOpt** - Req. Channel routing option. <br>
**udmpOpt** - Req. User-defined spatial mapping flag to turn on/off. <br>
**gwBaseSw** - Req. Groundwater option. <br>
**gwRestart** - Req. Flag to use restart states in groundwater scheme. <br>
**enableCompoundChannel** - Req. Flag to activate compound channel in the hydro.namelist. 1 - on, 0 - off. <br>
**compoundChannel** - Req. Activation flag for compound channel. enableCompoundChannel must be on. <br>
**enableGwBucketLoss** - Req. Flag to activate groundwater bucket loss in hydro.namelist. 1 - on, 0 - off. <br>
**bucket_loss** - Req. Activation flag for groundwater bucket loss. enableGwBucketLoss must be on. <br>

In [ ]:
%%bash
cd /home/docker/nwm-training/example_case/Calibration
cat setup.parm

## The calibration workflow

We will now navigate to where the calibration workflow code resides. At a top level, there are multiple programs that are used to run components of the calibration. 

In [ ]:
%%bash
cd /home/docker/nwm-training/PyWrfHydroCalib
ls

For our purposes, we will only be working with initDB.py, inputDomainMeta.py, jobInit.py, and calibOrchestrator.py. The initDB.py program initializes an sql-lite database file that contains the necessary tables to manage information about the workflow. The inputDomainMeta.py program will ingest information about our basin of interest, and place the data into the database tables. The jobInit.py program will ingest a configuration file we will create, place information about our experiment into tables, and create the necessary output directories where everything will be run. 

## Database initialization

The first step is to initialize the database file, which is simply a sqlite DB file. 

*IMPORTANT*: This file will contain all data on your workflow, including status values, parameter values, and error statistics. Be very careful when moving the file as once it's deleted, you cannot replicate it, unless you re-run the calibration experiment. 

The database file contains several tables. Some tables have metadata on the workflow, including paths to input files, domain information for each gage being calibrated. Other tables contain actual parameter values for each iteration during the calibration, along with the associated statistics. 

In [ ]:
%%bash
cd /home/docker/nwm-training/PyWrfHydroCalib
python initDB.py --help

In [ ]:
%%bash
cd /home/docker/nwm-training/PyWrfHydroCalib
python initDB.py
ls

## Domain information

This next step will utilize the .csv file we modified earlier containing information for the basin of interest, along with the inputDomainMeta.py program to ingest domain information into the database tables. The program will ensure all necessary input files are available, including the observations file.

In [ ]:
%%bash
cd /home/docker/nwm-training/PyWrfHydroCalib
python inputDomainMeta.py --help

In [ ]:
%%bash
cd /home/docker/nwm-training/PyWrfHydroCalib
python inputDomainMeta.py /home/docker/nwm-training/example_case/Calibration/domainMetaTemplate.csv

In Addition to top-level programs to execute the workflow components, there are utility programs for use to either print information to the screen for the user, or extract information about a calibration to an output file. The example below prints information about our domain we are calibrating to the screen for the user to verify it looks correct. 

In [ ]:
%%bash
cd /home/docker/nwm-training/PyWrfHydroCalib
python util/printDomainInfo.py wrfHydroCalib.db

## Experiment initialization

Below, we run the jobInit.py program to initialize our calibration experiment, and set everything up. It's worth noting this step does not actually initialize any model simulations, etc, but simply enters information into the database tables on what we are doing, along with creating all the output directories where results of our calibration will reside. 

Once the initialization has successfully completed, a message is returned that provides the user with a unique identification number for this particular experiment. Note that the user can include multiple domains across a region into one calibration experiment. This is specified in the configuration file through the SQL command. 

The ID that is returned is then used as an argument to remaining calibration programs to execute components (spinup,calibration,sensitivity analysis,validation).

In [ ]:
%%bash
cd /home/docker/nwm-training/PyWrfHydroCalib
python jobInit.py --help

In [ ]:
%%bash
cd /home/docker/nwm-training/PyWrfHydroCalib
python jobInit.py /home/docker/nwm-training/example_case/Calibration/setup.parm

Once again, we will be running one of the optional utility programs to ensure the details of our experiment look good. Note that we are passing the ID value returned to us in the previous step. It's possible the user may have multiple calibration experiments in one database file. 

In [ ]:
%%bash
cd /home/docker/nwm-training/PyWrfHydroCalib
python util/printJobMeta.py wrfHydroCalib.db 1

## Execution of calibration

Fortunately, the actual execution of the calibration is fairly simple. The only required argument is the ID number for our experiment. The workflow then handles the rest from there. Once the calibration is complete, a complete message is communicated back to the user. 

In [ ]:
%%bash
cd /home/docker/nwm-training/PyWrfHydroCalib
python calibOrchestrator.py --help

In [ ]:
%%bash
cd /home/docker/nwm-training/PyWrfHydroCalib
python calibOrchestrator.py 1

© UCAR 2019